


# **Georgia Disasters SAR HYDRAFloods Code**

*Authors: Shakirah Rogers, Nancee Uniyal, Isabella Chittumuri, Nathan Tesfayi*

*Project: Georgia Disasters, Fall 2022*

*Node: Athens, GA*

---
**Purpose**

This script uses the HYDRAFloods package to observe the flood extent of southern counties in Georgia from Hurricane Irma.

**Credit**

The HYDRAFloods tool and workshop materials used were created by Kel Markert https://github.com/Servir-Mekong/hydra-floods


# **1. Overview**
HYDRAFloods - Hydrologic Remote Sensing Analysis for Floods 

HYDRAFloods is a Google Earth Engine (GEE) with Python API based application developed by NASA SERVIR. The application uses satellite imagery to produce flood water maps which can aid in response to flood related disasters. 

This tutorial will walk-through the steps to apply HYDRAFloods to optical imagery to create flood water maps. For this example, we will examine 15 Georgia counties during Hurricanes Irma in September of 2017. 

Study Location: Berrien, Camden, Charlton, Chatham, Coffee, Cook, Crisp, Dougherty, Glynn, Liberty, McIntosh, Thomas, Turner, Wilcox, and Worth counties, GA

This tutorial was developed as part of the Georgia Disasters Project for the NASA DEVELOP Program during the Fall 2022 term.

In [ ]:
#Mexico Spring 2022 Gitlab https://gitlab.developprogram.org/john.willis/mexico-disasters/-/blob/main/HYDRAFloodsTutorial.md
#Hawaii Fall 2021 https://gitlab.developprogram.org/developcodecatalog/2021/fall-2021/hawaii_island_disasters

#compare to hawaii
#https://colab.research.google.com/drive/1hAEbazhMugfjzlq6t7huIM66FBPZDUU5?usp=sharing

# **2. Set-Up**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install hydrafloods geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# import packages
import ee
import datetime
import hydrafloods as hf
import geemap.foliumap as geemap

In [ ]:
_ = geemap.Map()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ZYkkY1WMVgzh8lAysVQ7IHhVUsYEBeIrndXNgI5HEG0&tc=t-3Fv-lUqtKPDNmU39LTcu3tEYngzt45a1jiniaISBI&cc=3wApfcXPmUI6XDX0xKot2W3MyXlDT8S4QLUV8cBAE50

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfgeXvutaH1VAJX3joi6t5PLV16BnlXGufR2ZzBHByCvTt3uEBapPXZnkN8

Successfully saved authorization token.


In [ ]:
#if the above is not prompting the GEE token, uncomment below
#ee.Authenticate()
#ee.Initialize()

# **3. Data Acquisition**

In [ ]:
#Region of Interest

#this shapefile has all the counties as separate attributes
#region = ee.FeatureCollection("projects/ee-tesfayinathan/assets/studyarea5").geometry()
#this shapefile has all the counties as one attribute that we can easily clip to
region = ee.FeatureCollection("projects/ee-tesfayinathan/assets/15counties").geometry()

In [ ]:
#Time Period When Flooding Occured

startTime = datetime.datetime(2017,9,11)
endTime = datetime.datetime(2017,9,20)

In [ ]:
# get SAR image collection filtered by above roi and time period parameters
s1 = hf.Sentinel1(region, startTime, endTime)

# print s1 image collection metadata and extract information
print('Metadata:', s1)
print('Number of Images in Collection:', s1.n_images)
print('Image Dates', s1.dates)

Metadata: HYDRAFloods Dataset:
{'asset_id': 'COPERNICUS/S1_GRD',
 'end_time': '2017-09-20',
 'name': 'Sentinel1',
 'region': [[[...]], [[...]], [[...]], [[...]]],
 'start_time': '2017-09-11'}
Number of Images in Collection: 9
Image Dates ['2017-09-15 23:37:24.000', '2017-09-15 23:37:49.000', '2017-09-16 11:41:01.000', '2017-09-16 11:41:26.000', '2017-09-17 23:21:08.000', '2017-09-17 23:21:33.000', '2017-09-18 11:24:45.000', '2017-09-18 11:25:14.000', '2017-09-12 11:24:22.000']


# **4. Processing and Analysis**

In [ ]:
#elevation datasets

#merit means Multi-Error-Removed Improved-Terrain, resolution 90m
#error components such as "absolute bias, stripe noise, speckle noise, and tree height bias" - http://hydro.iis.u-tokyo.ac.jp/~yamadai/MERIT_DEM/

merit = ee.Image("MERIT/Hydro/v1_0_1")

# extract DEM and HAND Bands
#HAND height above nearest drainage, relative to some nearby stream
hand = merit.select("hnd").unmask(0)

#changed the elevation to a dataset more relevant for our region by importing 3DEP
US_elev = ee.Image("USGS/3DEP/10m")
elevation = US_elev.select("elevation").unmask(0)


In [ ]:
# apply a pseudo-terrain flattenting algorithm to remove instances of shadow, 10 m is approx height of 3 story building
s1_flat = s1.apply_func(hf.slope_correction, elevation = US_elev, buffer = 10, model = "surface")

Terrain flattening is set to filter out 10m of shadow, which is the approx height of a 3 story building. "mask out slopes"

In [ ]:
# slope correction function documentation
#print(hf.slope_correction.__doc__)

In [ ]:
# apply the speckle filter algorithm to remove noise from the s1 data
s1_filtered = s1_flat.apply_func(hf.gamma_map)

In [ ]:
# mask all areas with elev. > 20 m above nearest water body
s1_lowelv = s1_filtered.apply_func(lambda x: x.updateMask(hand.lt(20)))

Edge Otsu Method from Markert et al., 2020; https://doi.org/10.3390/rs12152469

In [ ]:
# apply edge otsu (water/nonwater threshold) algorithm
water = s1_lowelv.apply_func(hf.edge_otsu, initial_threshold = -20, band = "VV", edge_buffer = 250, scale = 90)
#the initial threshold tells edge otsu where to begin looking for the inter-class variance
#edge buffer is used if the threshold misses something, features that are longer than this amount are classified as water
#scale is set to default, "scale to perform reduction operations"

#Question: it is unclear between the Merkert paper and the Mexico Gitlab if theyre conflating edge_buffer and edge_length

In [ ]:
#print(hf.edge_otsu.__doc__)
#gives documentation on what the edge otsu function does

Polarimetry (VV vs VH) described here; https://sentinels.copernicus.eu/web/sentinel/user-guides/sentinel-1-sar/product-overview/polarimetry

In [ ]:
# reduce image collection by mode
water_img = water.collection.reduce("mode").clip(region)

JRC: Jean-Francois Pekel, Andrew Cottam, Noel Gorelick, Alan S. Belward, High-resolution mapping of global surface water and its long-term changes. Nature 540, 418-422 (2016). (doi:10.1038/nature20584)

In [ ]:
# get the previous 5 years before study time of permenant water for context
permanent_water = (
    ee.ImageCollection("JRC/GSW1_2/YearlyHistory") # get the JRC historical dataset
    .filterDate("1985-01-01",endTime) # filter for historical data up to date of interest
    .limit(5, "system:time_start", False) # grab the 5 latest images
    .map(lambda x: x.select("waterClass").eq(3)) # extract out the permanent water class
    .sum() # check if a pixel has been classified as permanent water in the past 5 years
    .unmask(0)
    .gt(0)
).updateMask(water_img.gte(0)) # mask for only the water image we just calculated

In [ ]:
# find where the surface water image says there is water but not permanent water
s1floods = water_img.add(permanent_water).eq(1).selfMask()

# **5. SAR Map**

In [ ]:
# select coordinates to zoom in map on
Map = geemap.Map(center=(31.1856, -82.5001), zoom=9) 
#32.1656° N, 82.9001° W

#Map.add_layer(s1.collection.median(), {"bands": "VV", "min":-25, "max:":0}, 'Sentinel 1') # Aggregated (median) SAR imagery
#to clip to region
Map.add_layer(s1.collection.median().clip(region), {"bands": "VV", "min":-25, "max:":0}, 'Sentinel 1') # Aggregated (median) SAR imagery

Map.addLayer(permanent_water.selfMask(),{"min":0, "max":1,"palette": ["black", "blue"]}, 'Permanent Water Yearly') #Permanent Water
Map.addLayer(s1floods, {"min": 0, "max": 1, "palette": ["black", "red"]}, 'S1 Flood Extent') # Resulting Flood Extent Map
#Map.addLayer(region, {}, 'Regions Outline', bool = False)

Map.addLayerControl()
Map

# **6. SAR with Buffer**

Below is code from the Hawaii Disasters Project where they flooded out surrounding pixels w a buffer for missed flood from building or forest geometry

In [ ]:
# add a 20 m buffer around flooded pixels to assess areas impacted. 
# Change kernel type between "circle" or "square" and change the radius as needed
#final_flood_buffer = ee.Image.focalMax(image= s1floods, radius = 20, kernelType = 'circle', units = 'meters',iterations=1)


In [ ]:
# select coordinates to zoom in map on
#Map = geemap.Map(center=(31.1856, -82.5001), zoom=9) 

#Map.add_layer(s1.collection.median().clip(region), {"bands": "VV", "min":-25, "max:":0}, 'Sentinel 1') # Aggregated (median) SAR imagery

#Map.addLayer(permanent_water.selfMask(),{"min":0, "max":1,"palette": ["black", "blue"]}, 'Permanent Water Yearly') #Permanent Water
#Map.addLayer(s1floods, {"min": 0, "max": 1, "palette": ["black", "red"]}, 'S1 Flood Extent') # Resulting Flood Extent Map
#Map.addLayer(final_flood_buffer, {"min": 0, "max": 1, "palette": ["black", "red"]}, 'S1 Flood Extent + 20m buffer') # Buffered Flood Extent Map
#Map.addLayerControl()
#Map

In [ ]:
#need to add export code
#is this the right resolution?

In [ ]:
# Export flood map without buffer
#hf.export_image(
    #s1floods, 
    #region = region,
    #description = "s1 Flood Extent",
    #scale=10, # resolution of data, 10m
    #export_type='toDrive', # export to drive. can also export as 'Asset' to google earth engine
    #folder='Hydrafloods', # creates a folder called hydrafloods in google drive to save files in. Change the name or use an existing folder as needed
    #crs='EPSG:4326' 
#)

In [ ]:
task = ee.batch.Export.image.toDrive(image=s1floods,
                                     description='s1',
                                     scale=30,
                                     region= region,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
task.start()

In [ ]:
task.status()

In [ ]:
task = ee.batch.Export.image.toDrive(image=permanent_water,
                                     description='permanent_water_s1',
                                     scale=30,
                                     region= region,
                                     crs='EPSG:4326',
                                     fileFormat='GeoTIFF')
task.start()

In [ ]:
task.status()

In [ ]:
#task = ee.batch.Export.image.toDrive(image=final_flood_buffer,
#                                     description='s1_buffer',
#                                     scale=30,
#                                     region= region,
#                                     crs='EPSG:4326',
#                                     fileFormat='GeoTIFF')
#task.start()

In [ ]:
task.status()

In [ ]:
#floods without buffer
#hf.export_image(
#    s1floods, 
#    region.geometry(), 
#    description = "s1_Flood_Extent",
#    scale=10, 
#    crs='EPSG:4326', 
#    pyramiding={"water":"mode"}, 
#    export_type='toAsset',
#    #asset_id = "users/kelmarkert/public/hydrafloods_training_day2_water"
#)

# **7. Future Code**

Future code updates

1.   Observe seasonal approach to the permanent water from JRC Yearly Water Classification
2.   Floodwater Depth Estimation Tool, hf.fwdet
https://nhess.copernicus.org/articles/19/2053/2019/
3.   Could also do a time series to look at the flood from the first x days, 2nd x, etc (x being time step)
4. Buffer for flood measured with limited area


In [1]:
#this did not work
# apply the fwdet algorithm for the floods we extracted
#flood_depths = hf.fwdet(final_flood_buffer,US_elev,force_projection=True)

In [2]:
#a method to get the seasonal permanent water, but the flood extent is identical, which is why is why it wasn't used
#$permanent_water2 = (
#    ee.ImageCollection("JRC/GSW1_2/YearlyHistory") # get the JRC historical dataset
#    .filterDate("1985-01-01",endTime) # filter for historical data up to date of interest
#    .limit(5, "system:time_start", False) # grab the 5 latest images
#    .map(lambda x: x.select("waterClass").eq(2)) # extract out the seasonal water class
#    .sum() # check if a pixel has been classified as permanent water in the past 5 years
#    .unmask(0)
#    .gt(0)
#).updateMask(water_img2.gte(0)) # mask for only the water image we just calculated